In [2]:
!pip install tensorflow
!pip install scikit-learn
!pip install pandas
!pip install transformers==4.8.2
!pip install sentencepiece==0.1.96
!pip install tensorflow_addons
!pip install dill

In [3]:
#필요한거임포트
from IPython.display import clear_output 

clear_output()

#%tensorflow_version 1.x
import sklearn
import tensorflow
import transformers
import tensorflow_addons

print(sklearn.__version__) # 1.0.2
print(tensorflow.__version__) # 2.8.0
print(transformers.__version__) # 4.8.2
print(tensorflow_addons.__version__) # 0.16.1

import sentencepiece


import pandas as pd
import numpy as np  
import re
import pickle
import dill #함수를 파일로 저장하기 위해
import logging # 로그출력문과 관련된건듯
import urllib.request
from tqdm import tqdm

from sklearn import model_selection
from sklearn.metrics import accuracy_score


import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_addons as tfa #활성함수와 관련
from tensorflow.keras import layers, initializers, losses, optimizers, metrics, callbacks 

import transformers
from transformers import TFBertModel # BertTokenizer 제외

import sentencepiece as spm

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

import codecs
import shutil

2023-02-10 11:23:55.366653: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


1.2.1
2.9.1
4.8.2
0.19.0
Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
!pip install keras-bert
!pip install keras-radam

from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps
import os

from keras_radam import RAdam

#경로잘연결돼있는지확인
os.listdir('/root/project/bert')

['bert_model.ckpt.meta',
 'bert_config.json',
 'bert_model.ckpt.data-00000-of-00001',
 'vocab.txt',
 'bert_model.ckpt.index']

In [5]:
# Random seed 고정
#tf.random.set_seed(1234)
np.random.seed(1234)

# Transformers logging level 변경 (WARNING -> ERROR) @ https://huggingface.co/transformers/main_classes/logging.html
transformers.logging.set_verbosity(transformers.logging.ERROR)

# Tensorflow logging level 변경 
tf.get_logger().setLevel(logging.ERROR)

In [6]:
df_train=pd.read_csv("/root/project/dataset/감성대화말뭉치_Training.csv", encoding = "utf-8", header = 0, index_col = 0)
df_test=pd.read_csv("/root/project/dataset/감성대화말뭉치_Validation.csv", encoding = "utf-8", header = 0, index_col = 0)

In [7]:
df_train = df_train.drop_duplicates('문장')
df_train = df_train.reset_index(drop=True)
df_test = df_test.drop_duplicates('문장')
df_test = df_test.reset_index(drop=True)
#train 중복 29개행 제거됨.

#일단 대분류로만 진행해볼게요
emotion_map = {"불안":0,"분노":1,"상처":2,"슬픔":3,"당황":4,"기쁨":5}
df_train['label']= df_train['감정_대분류'].map(emotion_map)
df_test['label']= df_test['감정_대분류'].map(emotion_map)

In [8]:
df_train= df_train.loc[:,['문장','label']]
df_test= df_test.loc[:,['문장','label']]
#df_test.tail(4)

In [40]:
SEQ_LEN = 300 #문장의 최대 길이
BATCH_SIZE = 16 #배치 사이즈
EPOCHS=2
LR=1e-5

pretrained_path ="/root/project/bert"
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

DATA_COLUMN = "문장"
LABEL_COLUMN = "label"

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        if "_" in token:
          token = token.replace("_","")
          token = "##" + token
        token_dict[token] = len(token_dict)

In [41]:
class inherit_Tokenizer(Tokenizer):
  def _tokenize(self, text):
        if not self._cased:
            text = text
            
            text = text.lower()
        spaced = ''
        for ch in text:
            if self._is_punctuation(ch) or self._is_cjk_character(ch):
                spaced += ' ' + ch + ' '
            elif self._is_space(ch):
                spaced += ' '
            elif ord(ch) == 0 or ord(ch) == 0xfffd or self._is_control(ch):
                continue
            else:
                spaced += ch
        tokens = []
        for word in spaced.strip().split():
            tokens += self._word_piece_tokenize(word)
        return tokens

In [42]:
tokenizer = inherit_Tokenizer(token_dict)

#데이터프레임 넣어서 그걸로 토큰화
def convert_data(data_df):
    global tokenizer
    indices, targets = [], []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
        targets.append(data_df[LABEL_COLUMN][i])
    items = list(zip(indices, targets))
    
    indices, targets = zip(*items)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)], np.array(targets)

def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)


    data_x, data_y = convert_data(data_df)

    return data_x, data_y

In [43]:
train_x, train_y = load_data(df_train)
test_x, test_y = load_data(df_test)

100%|██████████| 6641/6641 [00:00<00:00, 14313.59it/s]


In [44]:
layer_num = 12
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,)

In [45]:
def get_bert_finetuning_model(model):
  inputs = model.inputs[:2]
  dense = model.layers[-3].output


  outputs = keras.layers.Dense(units=6, activation='softmax',
                               kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),name="classifier")(dense)

  bert_model = keras.models.Model(inputs, outputs)
  bert_model.compile(
      optimizer=RAdam(learning_rate=0.00001, weight_decay=0.0025),
      loss=losses.SparseCategoricalCrossentropy(),
      metrics=['accuracy'])
  
  return bert_model

In [46]:
sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init = tf.variables_initializer([v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables])
sess.run(init)

bert_model = get_bert_finetuning_model(model)
history = bert_model.fit(train_x, train_y, epochs=2, batch_size=16, verbose = 1, validation_data=(test_x, test_y), shuffle=True)

Train on 51601 samples, validate on 6641 samples
Epoch 1/2


2023-02-10 12:08:13.992659: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 56.25MiB (rounded to 58982400)requested by op Encoder-7-FeedForward_1/mul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-02-10 12:08:13.992891: I tensorflow/core/common_runtime/bfc_allocator.cc:1027] BFCAllocator dump for GPU_0_bfc
2023-02-10 12:08:13.992921: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (256): 	Total Chunks: 734, Chunks in use: 733. 183.5KiB allocated for chunks. 183.2KiB in use in bin. 3.2KiB client-requested in use in bin.
2023-02-10 12:08:13.992939: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-02-10 12:08:13.9929

ResourceExhaustedError: 2 root error(s) found.
  (0) RESOURCE_EXHAUSTED: failed to allocate memory
	 [[{{node Encoder-7-FeedForward_1/mul}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[metrics_2/acc/Identity/_5575]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

  (1) RESOURCE_EXHAUSTED: failed to allocate memory
	 [[{{node Encoder-7-FeedForward_1/mul}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

0 successful operations.
0 derived errors ignored.

In [18]:
path = '/root/project/'

#bert_model.save_weights(path+"/bert.h5")
bert_model = get_bert_finetuning_model(model)
bert_model.load_weights(path+"/bert.h5")


In [19]:
def predict_convert_data(data_df):
    global tokenizer
    indices = []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
        
    items = indices
    
    
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def predict_load_data(x): #Pandas Dataframe을 인풋으로 받는다
    data_df = x
    
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)


    data_x = predict_convert_data(data_df)

    return data_x

In [20]:
test_set = predict_load_data(df_test)
#예측
preds = bert_model.predict(test_set)

100%|██████████| 6641/6641 [00:00<00:00, 15011.13it/s]
/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-02-10 11:25:16.254604: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [21]:
preds

array([[0.02054891, 0.04301628, 0.06422771, 0.08375864, 0.7727116 ,
        0.01573691],
       [0.7908655 , 0.0351206 , 0.03063342, 0.02949227, 0.11086293,
        0.0030253 ],
       [0.69258684, 0.02949889, 0.02396376, 0.01553935, 0.23628436,
        0.00212684],
       ...,
       [0.05672211, 0.29497784, 0.39702547, 0.1325954 , 0.11719112,
        0.00148804],
       [0.06782398, 0.11195321, 0.31226203, 0.21304218, 0.28625834,
        0.00866028],
       [0.0917839 , 0.12004445, 0.3518719 , 0.1921522 , 0.23400585,
        0.01014169]], dtype=float32)

In [22]:
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix


preds = np.argmax(preds,axis=1) #그냥하니까안됨...
y_true = df_test['label']
# F1 Score 확인
print(classification_report(y_true, preds))

              precision    recall  f1-score   support

           0       0.66      0.67      0.67      1113
           1       0.72      0.64      0.68      1257
           2       0.48      0.58      0.53      1007
           3       0.57      0.63      0.59      1003
           4       0.65      0.60      0.62      1048
           5       0.93      0.85      0.89      1213

    accuracy                           0.67      6641
   macro avg       0.67      0.66      0.66      6641
weighted avg       0.68      0.67      0.67      6641



In [37]:
def sentence_convert_data(data):
    global tokenizer
    indices = []
    ids, segments = tokenizer.encode(data, max_len=SEQ_LEN)
    indices.append(ids)
        
    items = indices
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = bert_model.predict(data_x)
    predict_answer = np.argmax(np.ravel(predict))
    #print(predict_answer)
    if predict_answer == 0:
      return "불안"
    elif predict_answer == 1:
      return "분노"
    elif predict_answer ==2:
      return "상처"
    elif predict_answer ==3:
      return "슬픔"
    elif predict_answer ==4:
      return "당황"
    elif predict_answer ==5:
      return "기쁨"

In [38]:
#노래 리스트 불러오고 가공
song_list=pd.read_csv("/root/project/dataset/music_genre_total.csv", encoding = "utf-8", header = 0, index_col = 0)
song_list = song_list.iloc[:,1:]
#중복 노래 제거
song_list = song_list.drop_duplicates('title')
song_list = song_list.reset_index(drop=True)
nolyric = song_list[song_list['lyric'].isnull()]
#nolyric.genre.unique()
#포크 블루스 이런거 소수면 살리려 했는데 그냥 누락되어 보이는 데이터도 많으므로 쿨하게 드랍함
song_list = song_list.dropna(axis='index',how='any')
song_list

,title,artist,lyric,genre
0,내 번호를 지워줘,박수진,이제는 사랑 같은 거 다시는 안 할 거라며 헤어지던 날 다짐했었는데 마음처럼 쉽지 ...,발라드
1,빛 바래 사라지지만,마린,언젠가 함께 살자고 그저 바라보며 살면 된다고 몸 뉘일 작은방 한 칸과 그대만 있으...,"발라드, 인디음악"
2,설월화 (Vocal 이서한),그네,사시사철 좋은 풍경을 바라보며 해를 보냈죠 많은 걸 놓고 적은 걸 얻은 그런 해를 ...,"발라드, 인디음악"
3,작은별,아웃오브캠퍼스,너는 참 특별한 작은 별 작지만 선명하게 꼭 필요한 곳을 비추지 너는 참 용감한 작...,"발라드, 인디음악"
4,미안해,SOI,미안해 내가 날 많이 아껴준 너인데 후회하겠지만 이제는 말할게 미안해 정말 미안해 ...,"발라드, 인디음악"
...,...,...,...,...
11591,네가 바로 지구야,이성구,네가 바로 지구야 지구는 바로 너야 네가 바로 희망이야 희망은 바로 너야 네가 보는...,"인디음악, 포크/블루스"
11593,동화.1,조경윤,커피를 너무 마셔서 밤을 새운 어떤 새벽에 좋은 향기가 가득한 내 슬픔은 강아지들의...,"인디음악, 포크/블루스"
11594,"오늘의 계절 (Feat. 진예주, 윤준태)",팔레트 (Palette),나무들이 하나 둘씩 물들 때 우리는 추억에 물들었는데 이제는 나무들이 물들어도 추억...,포크/블루스
11595,캠퍼스,오느린윤혜린,자 이제 시작해 새로운 하루 아름다운 하늘을 좀 봐 저 푸른 세상엔 뭐가 있을까 알...,"인디음악, 포크/블루스"


In [39]:
song_list['emotion'] = song_list['lyric'].apply(evaluation_predict)
song_list

ValueError: Error when checking input: expected Input-Token to have shape (128,) but got array with shape (300,)

In [29]:
song_list.to_csv('music_total_labeled.csv', index=False)

In [35]:
song_list[song_list.emotion == '당황']

,title,artist,lyric,genre,emotion
6,Delusion (Feat. Chordgee),Hasilver (하실버),꿈에 네가 나왔어 그 날 하루 웬종일 기분이 그냥 좋아 원래 널 좋아헀나 네가 나왔...,발라드,당황
18,멍청이,임창정,그대 내 곁에 있었나요 기억이 나요 그렇게 지내던 날이 아직 보고 싶고 사랑하고 원...,발라드,당황
27,사랑하는 님이여,손헌식,사랑하는 당신이 보고 싶어도 그리워서 마음만 아파옵니다 저녁노을이 질 때 당신이 그...,발라드,당황
72,고백,우드,쉽게 넘을 수 없는 화단이 되어 그대에게 그대 마음에 또 큰 벽을 만들고 나의 마음...,"발라드, 인디음악",당황
108,추웠던 여름이였어,최민정,조금씩 무뎌져가는 날들에 정말 널 잊어서 그런건지 생각했어 그렇게도 사랑했던 우리였...,"발라드, 인디음악",당황
...,...,...,...,...,...
11519,사랑인거지,시인을 위하여,내 초롱한 오른쪽 현명한 눈빛은 엄마를 닮았고 늘 축 처져 피곤해 보이는 왼쪽 눈 ...,"인디음악, 포크/블루스",당황
11531,뻔한 진도는 스킵해,모어댄멜로디 (More Than Melody),어디까지 했더라 우리의 대화 서로 시작점은 달라도 목적지는 같잖아 너의 표정 눈빛 ...,"인디음악, 포크/블루스",당황
11541,맥주 한잔 합시다,정경일 원맨밴드,자 우리들 모두 오늘 여기에 모여 시원하게 맥주 한잔 합시다 잔을 가득채우고 거품 ...,포크/블루스,당황
11551,착각이었지!,나히 (Nahee),아마 일 년쯤 지났을걸 사실 걱정은 안 했어 딱히 그리워하지두 그치 우린 참 어렸지...,"인디음악, 포크/블루스",당황
